# Protein MD Setup tutorial using BioExcel Building Blocks (biobb) 
### --***AmberTools package version***--
**Based on the [MDWeb](http://mmb.irbbarcelona.org/MDWeb2/) [Amber FULL MD Setup tutorial](https://mmb.irbbarcelona.org/MDWeb2/help.php?id=workflows#AmberWorkflowFULL)**

***
This tutorial aims to illustrate the process of **setting up a simulation system** containing a **protein**, step by step, using the **BioExcel Building Blocks library (biobb)** wrapping the **AmberTools** utility from the **AMBER package**. The particular example used is the **Lysozyme** protein (PDB code 1AKI). 
***

## Settings

### Biobb modules used

 - [biobb_io](https://github.com/bioexcel/biobb_io): Tools to fetch biomolecular data from public databases.
 - [biobb_amber](https://github.com/bioexcel/biobb_amber): Tools to setup and run Molecular Dynamics simulations with AmberTools.
 - [biobb_analysis](https://github.com/bioexcel/biobb_analysis): Tools to analyse Molecular Dynamics trajectories.
 
### Auxiliar libraries used

 - [nb_conda_kernels](https://github.com/Anaconda-Platform/nb_conda_kernels): Enables a Jupyter Notebook or JupyterLab application in one conda environment to access kernels for Python, R, and other languages found in other environments.
 - [nglview](http://nglviewer.org/#nglview): Jupyter/IPython widget to interactively view molecular structures and trajectories in notebooks.
 - [ipywidgets](https://github.com/jupyter-widgets/ipywidgets): Interactive HTML widgets for Jupyter notebooks and the IPython kernel.
 - [plotly](https://plot.ly/python/offline/): Python interactive graphing library integrated in Jupyter notebooks.
 - [simpletraj](https://github.com/arose/simpletraj): Lightweight coordinate-only trajectory reader based on code from GROMACS, MDAnalysis and VMD.

### Conda Installation and Launch

```console
git clone https://github.com/bioexcel/biobb_wf_amber_md_setup.git
cd biobb_wf_amber_md_setup
conda env create -f conda_env/environment.yml
conda activate biobb_AMBER_MDsetup_tutorials
jupyter-nbextension enable --py --user widgetsnbextension
jupyter-nbextension enable --py --user nglview
jupyter-notebook biobb_wf_amber_md_setup/notebooks/mdsetup/biobb_amber_setup_notebook.ipynb
``` 

***
## Pipeline steps
 1. [Input Parameters](#input)
 2. [Fetching PDB Structure](#fetch)
 3. [Preparing PDB file for AMBER](#pdb4amber)
 4. [Create Protein System Topology](#top)
 5. [Energetically Minimize the Structure](#minv)
 6. [Create Solvent Box and Solvating the System](#box)
 7. [Adding Ions](#ions)
 8. [Energetically Minimize the System](#min)
 9. [Heating the System](#heating)
 10. [Equilibrate the System (NVT)](#nvt)
 11. [Equilibrate the System (NPT)](#npt)
 12. [Free Molecular Dynamics Simulation](#free)
 13. [Post-processing and Visualizing Resulting 3D Trajectory](#post)
 14. [Output Files](#output)
 15. [Questions & Comments](#questions)
 
***
<img src="https://bioexcel.eu/wp-content/uploads/2019/04/Bioexcell_logo_1080px_transp.png" alt="Bioexcel2 logo"
	title="Bioexcel2 logo" width="400" />
***


<a id="input"></a>
## Input parameters
**Input parameters** needed:
 - **pdbCode**: PDB code of the protein structure (e.g. 1AKI)

In [1]:
import nglview
import ipywidgets
import plotly
from plotly import subplots
import plotly.graph_objs as go

pdbCode = "1aki"

_ColormakerRegistry()

<a id="fetch"></a>
***
## Fetching PDB structure
Downloading **PDB structure** with the **protein molecule** from the RCSB PDB database.<br>
Alternatively, a **PDB file** can be used as starting structure. <br>
***
**Building Blocks** used:
 - [pdb](https://biobb-io.readthedocs.io/en/latest/api.html#module-api.pdb) from **biobb_io.api.pdb**
***

In [2]:
# Import module
from biobb_io.api.pdb import pdb

# Create properties dict and inputs/outputs
downloaded_pdb = pdbCode+'.pdb'

prop = {
    'pdb_code': pdbCode
}

#Create and launch bb
pdb(output_pdb_path=downloaded_pdb,
    properties=prop)

2021-06-21 16:56:10,480 [MainThread  ] [INFO ]  Downloading: 1aki from: https://www.ebi.ac.uk/pdbe/entry-files/download/pdb1aki.ent


/anaconda3/envs/biobb_dev_37/lib/python3.7/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ebi.ac.uk'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


2021-06-21 16:56:11,908 [MainThread  ] [INFO ]  Writting pdb to: 1aki.pdb
2021-06-21 16:56:11,910 [MainThread  ] [INFO ]  Filtering lines NOT starting with one of these words: ['ATOM', 'MODEL', 'ENDMDL']


0

<a id="vis3D"></a>
### Visualizing 3D structure
Visualizing the downloaded/given **PDB structure** using **NGL**:    

In [44]:
# Show protein
view = nglview.show_structure_file(downloaded_pdb)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','600px'])

view.render_image()
view.download_image(filename='ngl1.png')

view

NGLWidget()

<img src='ngl1.png'></img>

<a id="pdb4amber"></a>
***
## Preparing PDB file for AMBER
Before starting a **protein MD setup**, it is always strongly recommended to take a look at the initial structure and try to identify important **properties** and also possible **issues**. These properties and issues can be serious, as for example the definition of **disulfide bridges**, the presence of a **non-standard aminoacids** or **ligands**, or **missing residues**. Other **properties** and **issues** might not be so serious, but they still need to be addressed before starting the **MD setup process**. **Missing hydrogen atoms**, presence of **alternate atomic location indicators** or **inserted residue codes** (see [PDB file format specification](https://www.wwpdb.org/documentation/file-format-content/format33/sect9.html#ATOM)) are examples of these not so crucial characteristics. Please visit the [AMBER tutorial: Building Protein Systems in Explicit Solvent](http://ambermd.org/tutorials/basic/tutorial7/index.php) for more examples. **AmberTools** utilities from **AMBER MD package** contain a tool able to analyse **PDB files** and clean them for further usage, especially with the **AmberTools LEaP program**: the **pdb4amber tool**. The next step of the workflow is running this tool to analyse our **input PDB structure**.<br>

For the particular **Lysosyme** example, the most important property that is identified by the **pdb4amber** utility is the presence of **disulfide bridges** in the structure. Those are marked changing the residue names **from CYS to CYX**, which is the code that **AMBER force fields** use to distinguish between cysteines forming or not forming **disulfide bridges**. This will be used in the following step to correctly form a **bond** between these cysteine residues. 

We invite you to check what the tool does with different, more complex structures (e.g. PDB code [6N3V](https://www.rcsb.org/structure/6N3V)). 

***
**Building Blocks** used:
 - [pdb4amber_run](https://biobb-amber.readthedocs.io/en/latest/pdb4amber.html#pdb4amber-pdb4amber-run-module) from **biobb_amber.pdb4amber.pdb4amber_run**
***

In [4]:
# Import module
from biobb_amber.pdb4amber.pdb4amber_run import pdb4amber_run

# Create prop dict and inputs/outputs
output_pdb4amber_path = 'structure.pdb4amber.pdb'

# Create and launch bb
pdb4amber_run(input_pdb_path=downloaded_pdb,
            output_pdb_path=output_pdb4amber_path)

2021-06-21 16:56:29,781 [MainThread  ] [INFO ]  Creating 26d4edb7-bbf2-474a-99da-8a6dbbebbebd temporary folder
2021-06-21 16:56:29,782 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-06-21 16:56:30,567 [MainThread  ] [INFO ]  pdb4amber  -i 1aki.pdb -o structure.pdb4amber.pdb

2021-06-21 16:56:30,568 [MainThread  ] [INFO ]  Exit code 0

2021-06-21 16:56:30,569 [MainThread  ] [INFO ]  
Summary of pdb4amber for: 1aki.pdb

----------Chains
The following (original) chains have been found:
A

---------- Alternate Locations (Original Residues!))

The following residues had alternate locations:
None
-----------Non-standard-resnames


---------- Mising heavy atom(s)

None

2021-06-21 16:56:30,570 [MainThread  ] [INFO ]  Removed: 26d4edb7-bbf2-474a-99da-8a6dbbebbebd


0

<a id="top"></a>
***
## Create protein system topology
**Building AMBER topology** corresponding to the protein structure.<br>

*IMPORTANT: the previous pdb4amber building block is changing the proper cysteines residue naming in the PDB file from CYS to CYX so that this step can automatically identify and add the disulfide bonds to the system topology.*<br>

The **force field** used in this tutorial is [**ff14SB**](https://doi.org/10.1021/acs.jctc.5b00255), an evolution of the **ff99SB** force field with improved accuracy of protein side chains and backbone parameters. **Water** molecules type used in this tutorial is [**tip3p**](https://doi.org/10.1021/jp003020w).<br>
Adding **side chain atoms** and **hydrogen atoms** if missing. Forming **disulfide bridges** according to the info added in the previous step. <br>

Generating three output files: 
- **AMBER structure** (PDB file)
- **AMBER topology** (AMBER [Parmtop](https://ambermd.org/FileFormats.php#topology) file)
- **AMBER coordinates** (AMBER [Coordinate/Restart](https://ambermd.org/FileFormats.php#restart) file) 
***
**Building Blocks** used:
 - [leap_gen_top](https://biobb-amber.readthedocs.io/en/latest/leap.html#module-leap.leap_gen_top) from **biobb_amber.leap.leap_gen_top**
***

In [5]:
# Import module
from biobb_amber.leap.leap_gen_top import leap_gen_top

# Create prop dict and inputs/outputs
output_pdb_path = 'structure.leap.pdb'
output_top_path = 'structure.leap.top'
output_crd_path = 'structure.leap.crd'

prop = {
    "forcefield" : ["protein.ff14SB"]
}

# Create and launch bb
leap_gen_top(input_pdb_path=output_pdb4amber_path,
           #input_pdb_path=downloaded_pdb,
           output_pdb_path=output_pdb_path,
           output_top_path=output_top_path,
           output_crd_path=output_crd_path,
           properties=prop)

2021-06-21 16:56:34,267 [MainThread  ] [INFO ]  Creating fc81c8de-2dd1-4508-a307-d95b85fe5f6a temporary folder
2021-06-21 16:56:34,268 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-06-21 16:56:34,479 [MainThread  ] [INFO ]  tleap  -f fc81c8de-2dd1-4508-a307-d95b85fe5f6a/leap.in

2021-06-21 16:56:34,480 [MainThread  ] [INFO ]  Exit code 0

2021-06-21 16:56:34,481 [MainThread  ] [INFO ]  -I: Adding /anaconda3/envs/biobb_dev_37/dat/leap/prep to search path.
-I: Adding /anaconda3/envs/biobb_dev_37/dat/leap/lib to search path.
-I: Adding /anaconda3/envs/biobb_dev_37/dat/leap/parm to search path.
-I: Adding /anaconda3/envs/biobb_dev_37/dat/leap/cmd to search path.
-f: Source fc81c8de-2dd1-4508-a307-d95b85fe5f6a/leap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./fc81c8de-2dd1-4508-a307-d95b85fe5f6a/leap.in
----- Source: /anaconda3/envs/biobb_dev_37/dat/leap/cmd/leaprc.protein.ff14SB
----- Source of /anaconda3/envs/biobb_dev_37/dat/l

0

### Visualizing 3D structure
Visualizing the **PDB structure** using **NGL**. <br>
Try to identify the differences between the structure generated for the **system topology** and the **original one** (e.g. hydrogen atoms).   

In [45]:
# Show protein
view = nglview.show_structure_file(output_pdb_path)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','600px'])

view.render_image()
view.download_image(filename='ngl2.png')

view

NGLWidget()

<img src='ngl2.png'></img>

<a id="minv"></a>
## Energetically minimize the structure

**Energetically minimize** the **protein structure** (in vacuo) using the **sander tool** from the **AMBER MD package**. This step is **relaxing the structure**, usually **constrained**, especially when coming from an X-ray **crystal structure**. <br/>

The **miminization process** is done in two steps:
- [Step 1](#minv_1): **Hydrogen** minimization, applying **position restraints** (50 Kcal/mol.$Å^{2}$) to the **protein heavy atoms**.
- [Step 2](#minv_2): **System** minimization, applying **position restraints** (50 Kcal/mol.$Å^{2}$) to the **protein heavy atoms**.
***
**Building Blocks** used:
 - [sander_mdrun](https://biobb-amber.readthedocs.io/en/latest/sander.html#module-sander.sander_mdrun) from **biobb_amber.sander.sander_mdrun**
 - [process_minout](https://biobb-amber.readthedocs.io/en/latest/process.html#module-process.process_minout) from **biobb_amber.process.process_minout**
***

<a id="minv_1"></a>
### Step 1: Minimize Hydrogens
**Hydrogen** minimization, applying **position restraints** (50 Kcal/mol.$Å^{2}$) to the **protein heavy atoms**.

In [7]:
# Import module
from biobb_amber.sander.sander_mdrun import sander_mdrun

# Create prop dict and inputs/outputs
output_h_min_traj_path = 'sander.h_min.x'
output_h_min_rst_path = 'sander.h_min.rst'
output_h_min_log_path = 'sander.h_min.log'

prop = {
    'simulation_type' : "min_vacuo",
    "mdin" : { 
        'maxcyc' : 500,
        'ntpr' : 5,
        'ntr' : 1,
        'restraintmask' : '\":*&!@H=\"',
        'restraint_wt' : 50.0
    }
}

# Create and launch bb
sander_mdrun(input_top_path=output_top_path,
            input_crd_path=output_crd_path,
            input_ref_path=output_crd_path,
            output_traj_path=output_h_min_traj_path,
            output_rst_path=output_h_min_rst_path,
            output_log_path=output_h_min_log_path,
            properties=prop)

2021-06-21 16:56:39,598 [MainThread  ] [INFO ]  Creating 83f514a6-3a97-41b1-8dd8-16c553673503 temporary folder
2021-06-21 16:56:39,599 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-06-21 16:56:53,670 [MainThread  ] [INFO ]  sander -O -i 83f514a6-3a97-41b1-8dd8-16c553673503/sander.mdin -p structure.leap.top -c structure.leap.crd -r sander.h_min.rst -o sander.h_min.log -x sander.h_min.x -ref  structure.leap.crd

2021-06-21 16:56:53,671 [MainThread  ] [INFO ]  Exit code 0

2021-06-21 16:56:53,672 [MainThread  ] [INFO ]  Removed: mdinfo, 83f514a6-3a97-41b1-8dd8-16c553673503


0

### Checking Energy Minimization results
Checking **energy minimization** results. Plotting **potential energy** along time during the **minimization process**.

In [8]:
# Import module
from biobb_amber.process.process_minout import process_minout

# Create prop dict and inputs/outputs
output_h_min_dat_path = 'sander.h_min.energy.dat'

prop = {
    "terms" : ['ENERGY']
}

# Create and launch bb
process_minout(input_log_path=output_h_min_log_path,
            output_dat_path=output_h_min_dat_path,
            properties=prop)  

2021-06-21 16:56:57,347 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-06-21 16:56:57,415 [MainThread  ] [INFO ]  process_minout.perl  sander.h_min.log

2021-06-21 16:56:57,416 [MainThread  ] [INFO ]  Exit code 0

2021-06-21 16:56:57,417 [MainThread  ] [INFO ]  Processing sander output file (sander.h_min.log)...
Processing step 50 of a possible 500...
Processing step 100 of a possible 500...
Processing step 150 of a possible 500...
Processing step 200 of a possible 500...
Processing step 250 of a possible 500...
Processing step 300 of a possible 500...
Processing step 350 of a possible 500...
Processing step 400 of a possible 500...
Processing step 450 of a possible 500...
Processing step 500 of a possible 500...
Processing step 500 of a possible 500...
Starting output...
Outputing summary.NSTEP
Outputing summary.ENERGY
Outputing summary.RMS
Outputing summary.GMAX
Outputing summary.NAME
Outputing summary.NUMBER
Outputing summary.BOND
Outputi

0

In [9]:
#Read data from file and filter energy values higher than 1000 Kj/mol^-1
with open(output_h_min_dat_path,'r') as energy_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in energy_file 
            if not line.startswith(("#","@")) 
            if float(line.split()[1]) < 1000 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Energy Minimization",
                        xaxis=dict(title = "Energy Minimization Step"),
                        yaxis=dict(title = "Potential Energy kcal/mol")
                       )
}

plotly.offline.iplot(fig)

<a id="minv_2"></a>
### Step 2: Minimize the system
**System** minimization, applying **position restraints** (50 Kcal/mol.$Å^{2}$) to the **protein heavy atoms**.

In [10]:
# Import module
from biobb_amber.sander.sander_mdrun import sander_mdrun

# Create prop dict and inputs/outputs
output_n_min_traj_path = 'sander.n_min.x'
output_n_min_rst_path = 'sander.n_min.rst'
output_n_min_log_path = 'sander.n_min.log'

prop = {
    'simulation_type' : "min_vacuo",
    "mdin" : { 
        'maxcyc' : 500,
        'ntpr' : 5,
        'ntr' : 1,
        'restraintmask' : '\":*&!@H=\"',
        'restraint_wt' : 50.0
    }
}

# Create and launch bb
sander_mdrun(input_top_path=output_top_path,
            input_crd_path=output_h_min_rst_path,
            input_ref_path=output_h_min_rst_path,
            output_traj_path=output_n_min_traj_path,
            output_rst_path=output_n_min_rst_path,
            output_log_path=output_n_min_log_path,
            properties=prop)

2021-06-21 16:57:05,963 [MainThread  ] [INFO ]  Creating dc00d8c0-3b7b-43cc-9e2e-cd21acf8979f temporary folder
2021-06-21 16:57:05,965 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-06-21 16:57:19,544 [MainThread  ] [INFO ]  sander -O -i dc00d8c0-3b7b-43cc-9e2e-cd21acf8979f/sander.mdin -p structure.leap.top -c sander.h_min.rst -r sander.n_min.rst -o sander.n_min.log -x sander.n_min.x -ref  sander.h_min.rst

2021-06-21 16:57:19,545 [MainThread  ] [INFO ]  Exit code 0

2021-06-21 16:57:19,546 [MainThread  ] [INFO ]  Removed: mdinfo, dc00d8c0-3b7b-43cc-9e2e-cd21acf8979f


0

### Checking Energy Minimization results
Checking **energy minimization** results. Plotting **potential energy** by time during the **minimization process**.

In [11]:
# Import module
from biobb_amber.process.process_minout import process_minout

# Create prop dict and inputs/outputs
output_n_min_dat_path = 'sander.n_min.energy.dat'

prop = {
    "terms" : ['ENERGY']
}

# Create and launch bb
process_minout(input_log_path=output_n_min_log_path,
            output_dat_path=output_n_min_dat_path,
            properties=prop)

2021-06-21 16:57:25,882 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-06-21 16:57:25,933 [MainThread  ] [INFO ]  process_minout.perl  sander.n_min.log

2021-06-21 16:57:25,935 [MainThread  ] [INFO ]  Exit code 0

2021-06-21 16:57:25,936 [MainThread  ] [INFO ]  Processing sander output file (sander.n_min.log)...
Processing step 50 of a possible 500...
Processing step 100 of a possible 500...
Processing step 150 of a possible 500...
Processing step 200 of a possible 500...
Processing step 250 of a possible 500...
Processing step 300 of a possible 500...
Processing step 350 of a possible 500...
Processing step 400 of a possible 500...
Processing step 450 of a possible 500...
Processing step 500 of a possible 500...
Processing step 500 of a possible 500...
Starting output...
Outputing summary.NSTEP
Outputing summary.ENERGY
Outputing summary.RMS
Outputing summary.GMAX
Outputing summary.NAME
Outputing summary.NUMBER
Outputing summary.BOND
Outputi

0

In [12]:
#Read data from file and filter energy values higher than 1000 Kj/mol^-1
with open(output_n_min_dat_path,'r') as energy_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in energy_file 
            if not line.startswith(("#","@")) 
            if float(line.split()[1]) < 1000 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Energy Minimization",
                        xaxis=dict(title = "Energy Minimization Step"),
                        yaxis=dict(title = "Potential Energy kcal/mol")
                       )
}

plotly.offline.iplot(fig)

<a id="box"></a>
***
## Create solvent box and solvating the system
Define the unit cell for the **protein structure MD system** to fill it with water molecules.<br>
A **truncated octahedron box** is used to define the unit cell, with a **distance from the protein to the box edge of 9.0 Angstroms**. <br>
The solvent type used is the default **TIP3P** water model, a generic 3-point solvent model.
***
**Building Blocks** used:
 - [amber_to_pdb](https://biobb-amber.readthedocs.io/en/latest/ambpdb.html#module-ambpdb.amber_to_pdb) from **biobb_amber.ambpdb.amber_to_pdb**
 - [leap_solvate](https://biobb-amber.readthedocs.io/en/latest/leap.html#module-leap.leap_solvate) from **biobb_amber.leap.leap_solvate** 
***

### Getting minimized structure
Getting the result of the **energetic minimization** and converting it to **PDB format** to be then used as input for the **water box generation**. <br/>This is achieved by converting from **AMBER topology + coordinates** files to a **PDB file** using the **ambpdb** tool from the **AMBER MD package**.

In [13]:
# Import module
from biobb_amber.ambpdb.amber_to_pdb import amber_to_pdb

# Create prop dict and inputs/outputs
output_ambpdb_path = 'structure.ambpdb.pdb'

# Create and launch bb
amber_to_pdb(input_top_path=output_top_path,
            input_crd_path=output_h_min_rst_path,
            output_pdb_path=output_ambpdb_path
            )

2021-06-21 16:57:33,699 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-06-21 16:57:33,744 [MainThread  ] [INFO ]  ambpdb  -p structure.leap.top -c sander.h_min.rst >  structure.ambpdb.pdb

2021-06-21 16:57:33,746 [MainThread  ] [INFO ]  Exit code 0



0

### Create water box
Define the **unit cell** for the **protein structure MD system** and fill it with **water molecules**.<br/>

In [14]:
# Import module
from biobb_amber.leap.leap_solvate import leap_solvate

# Create prop dict and inputs/outputs
output_solv_pdb_path = 'structure.solv.pdb'
output_solv_top_path = 'structure.solv.parmtop'
output_solv_crd_path = 'structure.solv.crd'

prop = {
    "forcefield" : ["protein.ff14SB"],
    "water_type": "TIP3PBOX",
    "distance_to_molecule": "9.0",   
    "box_type": "truncated_octahedron"
}

# Create and launch bb
leap_solvate(input_pdb_path=output_ambpdb_path,
           output_pdb_path=output_solv_pdb_path,
           output_top_path=output_solv_top_path,
           output_crd_path=output_solv_crd_path,
           properties=prop)

2021-06-21 16:57:37,327 [MainThread  ] [INFO ]  Creating 98247fec-addc-4cda-9d81-79753d2f4eb9 temporary folder
2021-06-21 16:57:37,329 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-06-21 16:57:37,967 [MainThread  ] [INFO ]  tleap  -f 98247fec-addc-4cda-9d81-79753d2f4eb9/leap.in

2021-06-21 16:57:37,969 [MainThread  ] [INFO ]  Exit code 0

2021-06-21 16:57:37,969 [MainThread  ] [INFO ]  -I: Adding /anaconda3/envs/biobb_dev_37/dat/leap/prep to search path.
-I: Adding /anaconda3/envs/biobb_dev_37/dat/leap/lib to search path.
-I: Adding /anaconda3/envs/biobb_dev_37/dat/leap/parm to search path.
-I: Adding /anaconda3/envs/biobb_dev_37/dat/leap/cmd to search path.
-f: Source 98247fec-addc-4cda-9d81-79753d2f4eb9/leap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./98247fec-addc-4cda-9d81-79753d2f4eb9/leap.in
----- Source: /anaconda3/envs/biobb_dev_37/dat/leap/cmd/leaprc.protein.ff14SB
----- Source of /anaconda3/envs/biobb_dev_37/dat/l

0

<a id="ions"></a>
## Adding ions

**Neutralizing** the system and adding an additional **ionic concentration** using the **leap tool** from the **AMBER MD package**. <br/>
Using **Sodium (Na+)** and **Chloride (Cl-)** counterions and an **additional ionic concentration** of 150mM.
***
**Building Blocks** used:
 - [leap_add_ions](https://biobb-amber.readthedocs.io/en/latest/leap.html#module-leap.leap_add_ions) from **biobb_amber.leap.leap_add_ions**
***

In [15]:
# Import module
from biobb_amber.leap.leap_add_ions import leap_add_ions

# Create prop dict and inputs/outputs
output_ions_pdb_path = 'structure.ions.pdb'
output_ions_top_path = 'structure.ions.parmtop'
output_ions_crd_path = 'structure.ions.crd'

prop = {
    "forcefield" : ["protein.ff14SB"],
    "neutralise" : True,
    "positive_ions_type": "Na+",
    "negative_ions_type": "Cl-",
    "ionic_concentration" : 150, # 150mM
    "box_type": "truncated_octahedron"
}

# Create and launch bb
leap_add_ions(input_pdb_path=output_solv_pdb_path,
           output_pdb_path=output_ions_pdb_path,
           output_top_path=output_ions_top_path,
           output_crd_path=output_ions_crd_path,
           properties=prop)

2021-06-21 16:57:40,342 [MainThread  ] [INFO ]  Creating a2abf108-3ef6-467c-ba16-ae67836809c5 temporary folder
2021-06-21 16:57:40,429 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-06-21 16:57:41,539 [MainThread  ] [INFO ]  tleap  -f a2abf108-3ef6-467c-ba16-ae67836809c5/leap.in

2021-06-21 16:57:41,541 [MainThread  ] [INFO ]  Exit code 0

2021-06-21 16:57:41,541 [MainThread  ] [INFO ]  -I: Adding /anaconda3/envs/biobb_dev_37/dat/leap/prep to search path.
-I: Adding /anaconda3/envs/biobb_dev_37/dat/leap/lib to search path.
-I: Adding /anaconda3/envs/biobb_dev_37/dat/leap/parm to search path.
-I: Adding /anaconda3/envs/biobb_dev_37/dat/leap/cmd to search path.
-f: Source a2abf108-3ef6-467c-ba16-ae67836809c5/leap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./a2abf108-3ef6-467c-ba16-ae67836809c5/leap.in
----- Source: /anaconda3/envs/biobb_dev_37/dat/leap/cmd/leaprc.protein.ff14SB
----- Source of /anaconda3/envs/biobb_dev_37/dat/l

0

### Visualizing 3D structure
Visualizing the **protein system** with the newly added **solvent box** and **counterions** using **NGL**.<br> Note the **truncated octahedron box** filled with **water molecules** surrounding the **protein structure**, as well as the randomly placed **positive** (Na+, blue) and **negative** (Cl-, gray) **counterions**.

In [46]:
# Show protein
view = nglview.show_structure_file(output_ions_pdb_path)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein')
view.add_representation(repr_type='ball+stick', selection='solvent')
view.add_representation(repr_type='spacefill', selection='Cl- Na+')
view._remote_call('setSize', target='Widget', args=['','600px'])

view.render_image()
view.download_image(filename='ngl3.png')

view

NGLWidget()

<img src='ngl3.png'></img>

<a id="min"></a>
## Energetically minimize the system

**Energetically minimize** the **system** (protein structure + solvent + ions) using the **sander tool** from the **AMBER MD package**. **Restraining heavy atoms** with a force constant of 15 Kcal/mol.$Å^{2}$ to their initial positions.

- [Step 1](#emStep1): Energetically minimize the **system** through 500 minimization cycles.
- [Step 2](#emStep2): Checking **energy minimization** results. Plotting energy by time during the **minimization** process. 
***
**Building Blocks** used:
 - [sander_mdrun](https://biobb-amber.readthedocs.io/en/latest/sander.html#module-sander.sander_mdrun) from **biobb_amber.sander.sander_mdrun**
 - [process_minout](https://biobb-amber.readthedocs.io/en/latest/process.html#module-process.process_minout) from **biobb_amber.process.process_minout**
***

<a id="emStep1"></a>
### Step 1: Running Energy Minimization
The **minimization** type of the **simulation_type property** contains the main default parameters to run an **energy minimization**:

-  imin  = 1 ;    Minimization flag, perform an energy minimization.
-  maxcyc = 500;  The maximum number of cycles of minimization.
-  ntb = 1;       Periodic boundaries: constant volume.
-  ntmin = 2;     Minimization method: steepest descent.


In this particular example, the method used to run the **energy minimization** is the default **steepest descent**, with a **maximum number of 500 cycles** and **periodic conditions**. 

In [18]:
# Import module
from biobb_amber.sander.sander_mdrun import sander_mdrun

# Create prop dict and inputs/outputs
output_min_traj_path = 'sander.min.x'
output_min_rst_path = 'sander.min.rst'
output_min_log_path = 'sander.min.log'

prop = {
    "simulation_type" : "minimization",
    "mdin" : { 
        'maxcyc' : 300, # Reducing the number of minimization steps for the sake of time
        'ntr' : 1,      # Overwritting restrain parameter
        'restraintmask' : '\"!:WAT,Cl-,Na+\"',      # Restraining solute
        'restraint_wt' : 50.0                       # With a force constant of 50 Kcal/mol*A2
    }
}

# Create and launch bb
sander_mdrun(input_top_path=output_ions_top_path,
            input_crd_path=output_ions_crd_path,
            input_ref_path=output_ions_crd_path,
            output_traj_path=output_min_traj_path,
            output_rst_path=output_min_rst_path,
            output_log_path=output_min_log_path,
            properties=prop)

2021-06-21 16:57:56,844 [MainThread  ] [INFO ]  Creating a1738690-f827-44c6-9196-e6a30dc53667 temporary folder
2021-06-21 16:57:56,846 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-06-21 16:59:26,550 [MainThread  ] [INFO ]  sander -O -i a1738690-f827-44c6-9196-e6a30dc53667/sander.mdin -p structure.ions.parmtop -c structure.ions.crd -r sander.min.rst -o sander.min.log -x sander.min.x -ref  structure.ions.crd

2021-06-21 16:59:26,551 [MainThread  ] [INFO ]  Exit code 0

2021-06-21 16:59:26,553 [MainThread  ] [INFO ]  Removed: mdinfo, a1738690-f827-44c6-9196-e6a30dc53667


0

<a id="emStep2"></a>
### Step 2: Checking Energy Minimization results
Checking **energy minimization** results. Plotting **potential energy** along time during the **minimization process**.

In [19]:
# Import module
from biobb_amber.process.process_minout import process_minout

# Create prop dict and inputs/outputs
output_dat_path = 'sander.min.energy.dat'

prop = {
    "terms" : ['ENERGY']
}

# Create and launch bb
process_minout(input_log_path=output_min_log_path,
            output_dat_path=output_dat_path,
            properties=prop)

2021-06-21 16:59:30,762 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-06-21 16:59:30,788 [MainThread  ] [INFO ]  process_minout.perl  sander.min.log

2021-06-21 16:59:30,789 [MainThread  ] [INFO ]  Exit code 0

2021-06-21 16:59:30,790 [MainThread  ] [INFO ]  Processing sander output file (sander.min.log)...
Processing step 150 of a possible 300...
Processing step 300 of a possible 300...
Processing step 300 of a possible 300...
Starting output...
Outputing summary.NSTEP
Outputing summary.ENERGY
Outputing summary.RMS
Outputing summary.GMAX
Outputing summary.NAME
Outputing summary.NUMBER
Outputing summary.BOND
Outputing summary.ANGLE
Outputing summary.DIHEDRAL
Outputing summary.VDWAALS
Outputing summary.EEL
Outputing summary.HBOND
Outputing summary.VDW14
Outputing summary.EEL14
Outputing summary.RESTRAINT

2021-06-21 16:59:30,794 [MainThread  ] [INFO ]  Removed: [PosixPath('summary.DIHEDRAL'), PosixPath('summary.NUMBER'), PosixPath('summary.B

0

In [20]:
#Read data from file and filter energy values higher than 1000 Kj/mol^-1
with open(output_dat_path,'r') as energy_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in energy_file 
            if not line.startswith(("#","@")) 
            if float(line.split()[1]) < 1000 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Energy Minimization",
                        xaxis=dict(title = "Energy Minimization Step"),
                        yaxis=dict(title = "Potential Energy kcal/mol")
                       )
}

plotly.offline.iplot(fig)

<a id="heating"></a>
## Heating the system

**Warming up** the **prepared system** using the **sander tool** from the **AMBER MD package**. Going from 0 to the desired **temperature**, in this particular example, 300K. **Solute atoms restrained** (force constant of 10 Kcal/mol). Length 5ps.
***
- [Step 1](#heatStep1): Warming up the **system** through 500 MD steps.
- [Step 2](#heatStep2): Checking results for the **system warming up**. Plotting **temperature** along time during the **heating** process. 
***
**Building Blocks** used:
 - [sander_mdrun](https://biobb-amber.readthedocs.io/en/latest/sander.html#module-sander.sander_mdrun) from **biobb_amber.sander.sander_mdrun**
 - [process_mdout](https://biobb-amber.readthedocs.io/en/latest/process.html#module-process.process_mdout) from **biobb_amber.process.process_mdout**
***

<a id="heatStep1"></a>
### Step 1: Warming up the system
The **heat** type of the **simulation_type property** contains the main default parameters to run a **system warming up**:

-  imin = 0;&nbsp;&nbsp;&nbsp;      Run MD (no minimization)
-  ntx = 5;&nbsp;&nbsp;&nbsp;       Read initial coords and vels from restart file
-  cut = 10.0;&nbsp;&nbsp;&nbsp;    Cutoff for non bonded interactions in Angstroms
-  ntr = 0;&nbsp;&nbsp;&nbsp;       No restrained atoms
-  ntc = 2;&nbsp;&nbsp;&nbsp;       SHAKE for constraining length of bonds involving Hydrogen atoms
-  ntf = 2;&nbsp;&nbsp;&nbsp;       Bond interactions involving H omitted
-  ntt = 3;&nbsp;&nbsp;&nbsp;       Constant temperature using Langevin dynamics
-  ig = -1;&nbsp;&nbsp;&nbsp;       Seed for pseudo-random number generator
-  ioutfm = 1;&nbsp;&nbsp;&nbsp;    Write trajectory in netcdf format
-  iwrap = 1;&nbsp;&nbsp;&nbsp;     Wrap coords into primary box
-  nstlim = 5000;&nbsp;&nbsp;&nbsp; Number of MD steps 
-  dt = 0.002;&nbsp;&nbsp;&nbsp;    Time step (in ps)
-  tempi = 0.0;&nbsp;&nbsp;&nbsp;   Initial temperature (0 K)
-  temp0 = 300.0;&nbsp;&nbsp;&nbsp; Final temperature (300 K)
-  irest = 0;&nbsp;&nbsp;&nbsp;     No restart from previous simulation
-  ntb = 1;&nbsp;&nbsp;&nbsp;       Periodic boundary conditions at constant volume
-  gamma_ln = 1.0;&nbsp;&nbsp;&nbsp;   Collision frequency for Langevin dynamics (in 1/ps)

In this particular example, the **heating** of the system is done in **2500 steps** (5ps) and is going **from 0K to 300K** (note that the number of steps has been reduced in this tutorial, for the sake of time). 

In [21]:
# Import module
from biobb_amber.sander.sander_mdrun import sander_mdrun

# Create prop dict and inputs/outputs
output_heat_traj_path = 'sander.heat.netcdf'
output_heat_rst_path = 'sander.heat.rst'
output_heat_log_path = 'sander.heat.log'

prop = {
    "simulation_type" : "heat",
    "mdin" : { 
        'nstlim' : 2500, # Reducing the number of steps for the sake of time (5ps)
        'ntr' : 1,       # Overwritting restrain parameter
        'restraintmask' : '\"!:WAT,Cl-,Na+\"',      # Restraining solute
        'restraint_wt' : 10.0                       # With a force constant of 10 Kcal/mol*A2
    }
}

# Create and launch bb
sander_mdrun(input_top_path=output_ions_top_path,
            input_crd_path=output_min_rst_path,
            input_ref_path=output_min_rst_path,
            output_traj_path=output_heat_traj_path,
            output_rst_path=output_heat_rst_path,
            output_log_path=output_heat_log_path,
            properties=prop)

2021-06-21 16:59:36,436 [MainThread  ] [INFO ]  Creating fc4ae585-76ee-4f28-a7f4-9a1e85d082d9 temporary folder
2021-06-21 16:59:36,439 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-06-21 17:15:58,719 [MainThread  ] [INFO ]  sander -O -i fc4ae585-76ee-4f28-a7f4-9a1e85d082d9/sander.mdin -p structure.ions.parmtop -c sander.min.rst -r sander.heat.rst -o sander.heat.log -x sander.heat.netcdf -ref  sander.min.rst

2021-06-21 17:15:58,720 [MainThread  ] [INFO ]  Exit code 0

2021-06-21 17:15:58,721 [MainThread  ] [INFO ]  Removed: mdinfo, fc4ae585-76ee-4f28-a7f4-9a1e85d082d9


0

<a id="heatStep2"></a>
### Step 2: Checking results from the system warming up
Checking **system warming up** output. Plotting **temperature** along time during the **heating process**.

In [24]:
# Import module
from biobb_amber.process.process_mdout import process_mdout

# Create prop dict and inputs/outputs
output_dat_heat_path = 'sander.md.temp.dat'

prop = {
    "terms" : ['TEMP']
}

# Create and launch bb
process_mdout(input_log_path=output_heat_log_path,
            output_dat_path=output_dat_heat_path,
            properties=prop)

2021-06-21 17:17:49,910 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-06-21 17:17:49,932 [MainThread  ] [INFO ]  process_mdout.perl  sander.heat.log

2021-06-21 17:17:49,934 [MainThread  ] [INFO ]  Exit code 0

2021-06-21 17:17:49,935 [MainThread  ] [INFO ]  Processing sander output file (sander.heat.log)...
Starting output...
Outputing summary.TEMP
Outputing summary_avg.TEMP
Outputing summary_rms.TEMP
Outputing summary.TSOLUTE
Outputing summary_avg.TSOLUTE
Outputing summary_rms.TSOLUTE
Outputing summary.TSOLVENT
Outputing summary_avg.TSOLVENT
Outputing summary_rms.TSOLVENT
Outputing summary.PRES
Outputing summary_avg.PRES
Outputing summary_rms.PRES
Outputing summary.EKCMT
Outputing summary_avg.EKCMT
Outputing summary_rms.EKCMT
Outputing summary.ETOT
Outputing summary_avg.ETOT
Outputing summary_rms.ETOT
Outputing summary.EKTOT
Outputing summary_avg.EKTOT
Outputing summary_rms.EKTOT
Outputing summary.EPTOT
Outputing summary_avg.EPTOT
Outputi

0

In [25]:
#Read data from file and filter energy values higher than 1000 Kj/mol^-1
with open(output_dat_heat_path,'r') as energy_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in energy_file 
            if not line.startswith(("#","@")) 
            if float(line.split()[1]) < 1000 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Heating process",
                        xaxis=dict(title = "Heating Step (ps)"),
                        yaxis=dict(title = "Temperature (K)")
                       )
}

plotly.offline.iplot(fig)

<a id="nvt"></a>
***
## Equilibrate the system (NVT)
Equilibrate the **protein system** in **NVT ensemble** (constant Number of particles, Volume and Temperature). Protein **heavy atoms** will be restrained using position restraining forces: movement is permitted, but only after overcoming a substantial energy penalty. The utility of position restraints is that they allow us to equilibrate our solvent around our protein, without the added variable of structural changes in the protein.

- [Step 1](#eqNVTStep1): Equilibrate the **protein system** with **NVT** ensemble.
- [Step 2](#eqNVTStep2): Checking **NVT Equilibration** results. Plotting **system temperature** by time during the **NVT equilibration** process.  
***
**Building Blocks** used:
 - [sander_mdrun](https://biobb-amber.readthedocs.io/en/latest/sander.html#module-sander.sander_mdrun) from **biobb_amber.sander.sander_mdrun**
 - [process_mdout](https://biobb-amber.readthedocs.io/en/latest/process.html#module-process.process_mdout) from **biobb_amber.process.process_mdout** 
***

<a id="eqNVTStep1"></a>
### Step 1: Equilibrating the system (NVT)
The **nvt** type of the **simulation_type property** contains the main default parameters to run a **system equilibration in NVT ensemble**:

-  imin = 0;&nbsp;&nbsp;&nbsp;      Run MD (no minimization)
-  ntx = 5;&nbsp;&nbsp;&nbsp;       Read initial coords and vels from restart file
-  cut = 10.0;&nbsp;&nbsp;&nbsp;    Cutoff for non bonded interactions in Angstroms
-  ntr = 0;&nbsp;&nbsp;&nbsp;       No restrained atoms
-  ntc = 2;&nbsp;&nbsp;&nbsp;       SHAKE for constraining length of bonds involving Hydrogen atoms
-  ntf = 2;&nbsp;&nbsp;&nbsp;       Bond interactions involving H omitted
-  ntt = 3;&nbsp;&nbsp;&nbsp;       Constant temperature using Langevin dynamics
-  ig = -1;&nbsp;&nbsp;&nbsp;       Seed for pseudo-random number generator
-  ioutfm = 1;&nbsp;&nbsp;&nbsp;    Write trajectory in netcdf format
-  iwrap = 1;&nbsp;&nbsp;&nbsp;     Wrap coords into primary box
-  nstlim = 5000;&nbsp;&nbsp;&nbsp; Number of MD steps 
-  dt = 0.002;&nbsp;&nbsp;&nbsp;    Time step (in ps)
-  irest = 1;&nbsp;&nbsp;&nbsp;     Restart previous simulation
-  ntb = 1;&nbsp;&nbsp;&nbsp;       Periodic boundary conditions at constant volume
-  gamma_ln = 5.0;&nbsp;&nbsp;&nbsp;   Collision frequency for Langevin dynamics (in 1/ps)

In this particular example, the **NVT equilibration** of the system is done in **500 steps** (note that the number of steps has been reduced in this tutorial, for the sake of time). 

In [26]:
# Import module
from biobb_amber.sander.sander_mdrun import sander_mdrun

# Create prop dict and inputs/outputs
output_nvt_traj_path = 'sander.nvt.netcdf'
output_nvt_rst_path = 'sander.nvt.rst'
output_nvt_log_path = 'sander.nvt.log'

prop = {
    "simulation_type" : 'nvt',
    "mdin" : { 
        'nstlim' : 500, # Reducing the number of steps for the sake of time (1ps)
        'ntr' : 1,      # Overwritting restrain parameter
        'restraintmask' : '\"!:WAT,Cl-,Na+ & !@H=\"',      # Restraining solute heavy atoms
        'restraint_wt' : 5.0                               # With a force constant of 5 Kcal/mol*A2
    }
}

# Create and launch bb
sander_mdrun(input_top_path=output_ions_top_path,
            input_crd_path=output_heat_rst_path,
            input_ref_path=output_heat_rst_path,
            output_traj_path=output_nvt_traj_path,
            output_rst_path=output_nvt_rst_path,
            output_log_path=output_nvt_log_path,
            properties=prop)

2021-06-21 17:17:55,063 [MainThread  ] [INFO ]  Creating 4ce682b0-527b-485e-9b91-0a32d53ed89a temporary folder
2021-06-21 17:17:55,064 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-06-21 17:21:34,752 [MainThread  ] [INFO ]  sander -O -i 4ce682b0-527b-485e-9b91-0a32d53ed89a/sander.mdin -p structure.ions.parmtop -c sander.heat.rst -r sander.nvt.rst -o sander.nvt.log -x sander.nvt.netcdf -ref  sander.heat.rst

2021-06-21 17:21:34,755 [MainThread  ] [INFO ]  Exit code 0

2021-06-21 17:21:34,756 [MainThread  ] [INFO ]  Removed: mdinfo, 4ce682b0-527b-485e-9b91-0a32d53ed89a


0

<a id="eqNVTStep2"></a>
### Step 2: Checking NVT Equilibration results
Checking **NVT Equilibration** results. Plotting **system temperature** by time during the NVT equilibration process. 

In [27]:
# Import module
from biobb_amber.process.process_mdout import process_mdout

# Create prop dict and inputs/outputs
output_dat_nvt_path = 'sander.md.nvt.temp.dat'

prop = {
    "terms" : ['TEMP']
}

# Create and launch bb
process_mdout(input_log_path=output_nvt_log_path,
            output_dat_path=output_dat_nvt_path,
            properties=prop)

2021-06-21 17:21:34,769 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-06-21 17:21:34,791 [MainThread  ] [INFO ]  process_mdout.perl  sander.nvt.log

2021-06-21 17:21:34,792 [MainThread  ] [INFO ]  Exit code 0

2021-06-21 17:21:34,793 [MainThread  ] [INFO ]  Processing sander output file (sander.nvt.log)...
Starting output...
Outputing summary.TEMP
Outputing summary_avg.TEMP
Outputing summary_rms.TEMP
Outputing summary.TSOLUTE
Outputing summary_avg.TSOLUTE
Outputing summary_rms.TSOLUTE
Outputing summary.TSOLVENT
Outputing summary_avg.TSOLVENT
Outputing summary_rms.TSOLVENT
Outputing summary.PRES
Outputing summary_avg.PRES
Outputing summary_rms.PRES
Outputing summary.EKCMT
Outputing summary_avg.EKCMT
Outputing summary_rms.EKCMT
Outputing summary.ETOT
Outputing summary_avg.ETOT
Outputing summary_rms.ETOT
Outputing summary.EKTOT
Outputing summary_avg.EKTOT
Outputing summary_rms.EKTOT
Outputing summary.EPTOT
Outputing summary_avg.EPTOT
Outputing

0

In [31]:
#Read data from file and filter energy values higher than 1000 Kj/mol^-1
with open(output_dat_nvt_path,'r') as energy_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in energy_file 
            if not line.startswith(("#","@")) 
            if float(line.split()[1]) < 1000 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="NVT equilibration",
                        xaxis=dict(title = "Equilibration Step (ps)"),
                        yaxis=dict(title = "Temperature (K)")
                       )
}

plotly.offline.iplot(fig)

<a id="npt"></a>
***
## Equilibrate the system (NPT)
Equilibrate the **protein system** in **NPT ensemble** (constant Number of particles, Pressure and Temperature). Protein **heavy atoms** will be restrained using position restraining forces: movement is permitted, but only after overcoming a substantial energy penalty. The utility of position restraints is that they allow us to equilibrate our solvent around our protein, without the added variable of structural changes in the protein.

- [Step 1](#eqNPTStep1): Equilibrate the **protein system** with **NPT** ensemble.
- [Step 2](#eqNPTStep2): Checking **NPT Equilibration** results. Plotting **system pressure and density** by time during the **NVT equilibration** process.  
***
**Building Blocks** used:
 - [sander_mdrun](https://biobb-amber.readthedocs.io/en/latest/sander.html#module-sander.sander_mdrun) from **biobb_amber.sander.sander_mdrun**
 - [process_mdout](https://biobb-amber.readthedocs.io/en/latest/process.html#module-process.process_mdout) from **biobb_amber.process.process_mdout** 
***

<a id="eqNPTStep1"></a>
### Step 1: Equilibrating the system (NPT)
The **npt** type of the **simulation_type property** contains the main default parameters to run a **system equilibration in NPT ensemble**:

-  imin = 0;&nbsp;&nbsp;&nbsp;      Run MD (no minimization)
-  ntx = 5;&nbsp;&nbsp;&nbsp;       Read initial coords and vels from restart file
-  cut = 10.0;&nbsp;&nbsp;&nbsp;    Cutoff for non bonded interactions in Angstroms
-  ntr = 0;&nbsp;&nbsp;&nbsp;       No restrained atoms
-  ntc = 2;&nbsp;&nbsp;&nbsp;       SHAKE for constraining length of bonds involving Hydrogen atoms
-  ntf = 2;&nbsp;&nbsp;&nbsp;       Bond interactions involving H omitted
-  ntt = 3;&nbsp;&nbsp;&nbsp;       Constant temperature using Langevin dynamics
-  ig = -1;&nbsp;&nbsp;&nbsp;       Seed for pseudo-random number generator
-  ioutfm = 1;&nbsp;&nbsp;&nbsp;    Write trajectory in netcdf format
-  iwrap = 1;&nbsp;&nbsp;&nbsp;     Wrap coords into primary box
-  nstlim = 5000;&nbsp;&nbsp;&nbsp; Number of MD steps 
-  dt = 0.002;&nbsp;&nbsp;&nbsp;    Time step (in ps)
-  irest = 1;&nbsp;&nbsp;&nbsp;     Restart previous simulation
-  gamma_ln = 5.0;&nbsp;&nbsp;&nbsp;   Collision frequency for Langevin dynamics (in 1/ps)
-  pres0 = 1.0;&nbsp;&nbsp;&nbsp;   Reference pressure
-  ntp = 1;&nbsp;&nbsp;&nbsp;       Constant pressure dynamics: md with isotropic position scaling
-  taup = 2.0;&nbsp;&nbsp;&nbsp;    Pressure relaxation time (in ps)

In this particular example, the **NPT equilibration** of the system is done in **500 steps** (note that the number of steps has been reduced in this tutorial, for the sake of time). 

In [32]:
# Import module
from biobb_amber.sander.sander_mdrun import sander_mdrun

# Create prop dict and inputs/outputs
output_npt_traj_path = 'sander.npt.netcdf'
output_npt_rst_path = 'sander.npt.rst'
output_npt_log_path = 'sander.npt.log'

prop = {
    "simulation_type" : 'npt',
    "mdin" : { 
        'nstlim' : 500, # Reducing the number of steps for the sake of time (1ps)
        'ntr' : 1,      # Overwritting restrain parameter
        'restraintmask' : '\"!:WAT,Cl-,Na+ & !@H=\"',      # Restraining solute heavy atoms
        'restraint_wt' : 2.5                               # With a force constant of 2.5 Kcal/mol*A2
    }
}

# Create and launch bb
sander_mdrun(input_top_path=output_ions_top_path,
            input_crd_path=output_nvt_rst_path,
            input_ref_path=output_nvt_rst_path,
            output_traj_path=output_npt_traj_path,
            output_rst_path=output_npt_rst_path,
            output_log_path=output_npt_log_path,
            properties=prop)

2021-06-21 17:22:17,949 [MainThread  ] [INFO ]  Creating 9ddb15e6-b28b-47cc-9dba-e4fbe6b6dad9 temporary folder
2021-06-21 17:22:17,950 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-06-21 17:25:46,714 [MainThread  ] [INFO ]  sander -O -i 9ddb15e6-b28b-47cc-9dba-e4fbe6b6dad9/sander.mdin -p structure.ions.parmtop -c sander.nvt.rst -r sander.npt.rst -o sander.npt.log -x sander.npt.netcdf -ref  sander.nvt.rst

2021-06-21 17:25:46,715 [MainThread  ] [INFO ]  Exit code 0

2021-06-21 17:25:46,716 [MainThread  ] [INFO ]  Removed: mdinfo, 9ddb15e6-b28b-47cc-9dba-e4fbe6b6dad9


0

<a id="eqNPTStep2"></a>
### Step 2: Checking NPT Equilibration results
Checking **NPT Equilibration** results. Plotting **system pressure and density** by time during the **NPT equilibration** process. 

In [33]:
# Import module
from biobb_amber.process.process_mdout import process_mdout

# Create prop dict and inputs/outputs
output_dat_npt_path = 'sander.md.npt.dat'

prop = {
    "terms" : ['PRES','DENSITY']
}

# Create and launch bb
process_mdout(input_log_path=output_npt_log_path,
            output_dat_path=output_dat_npt_path,
            properties=prop)

2021-06-21 17:25:46,727 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-06-21 17:25:46,750 [MainThread  ] [INFO ]  process_mdout.perl  sander.npt.log

2021-06-21 17:25:46,752 [MainThread  ] [INFO ]  Exit code 0

2021-06-21 17:25:46,753 [MainThread  ] [INFO ]  Processing sander output file (sander.npt.log)...
Starting output...
Outputing summary.TEMP
Outputing summary_avg.TEMP
Outputing summary_rms.TEMP
Outputing summary.TSOLUTE
Outputing summary_avg.TSOLUTE
Outputing summary_rms.TSOLUTE
Outputing summary.TSOLVENT
Outputing summary_avg.TSOLVENT
Outputing summary_rms.TSOLVENT
Outputing summary.PRES
Outputing summary_avg.PRES
Outputing summary_rms.PRES
Outputing summary.EKCMT
Outputing summary_avg.EKCMT
Outputing summary_rms.EKCMT
Outputing summary.ETOT
Outputing summary_avg.ETOT
Outputing summary_rms.ETOT
Outputing summary.EKTOT
Outputing summary_avg.EKTOT
Outputing summary_rms.EKTOT
Outputing summary.EPTOT
Outputing summary_avg.EPTOT
Outputing

0

In [34]:
# Read pressure and density data from file 
with open(output_dat_npt_path,'r') as pd_file:
    x,y,z = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]),float(line.split()[2]))
            for line in pd_file 
            if not line.startswith(("#","@")) 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

trace1 = go.Scatter(
    x=x,y=y
)
trace2 = go.Scatter(
    x=x,y=z
)

fig = subplots.make_subplots(rows=1, cols=2, print_grid=False)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)

fig['layout']['xaxis1'].update(title='Time (ps)')
fig['layout']['xaxis2'].update(title='Time (ps)')
fig['layout']['yaxis1'].update(title='Pressure (bar)')
fig['layout']['yaxis2'].update(title='Density (Kg*m^-3)')

fig['layout'].update(title='Pressure and Density during NPT Equilibration')
fig['layout'].update(showlegend=False)

plotly.offline.iplot(fig)

<a id="free"></a>
***
## Free Molecular Dynamics Simulation
Upon completion of the **two equilibration phases (NVT and NPT)**, the system is now well-equilibrated at the desired temperature and pressure. The **position restraints** can now be released. The last step of the **protein** MD setup is a short, **free MD simulation**, to ensure the robustness of the system. 
- [Step 1](#mdStep1): Run short MD simulation of the **protein system**.
- [Step 2](#mdStep2): Checking results for the final step of the setup process, the **free MD run**. Plotting **Root Mean Square deviation (RMSd)** and **Radius of Gyration (Rgyr)** by time during the **free MD run** step.
***
**Building Blocks** used:
 - [sander_mdrun](https://biobb-amber.readthedocs.io/en/latest/sander.html#module-sander.sander_mdrun) from **biobb_amber.sander.sander_mdrun**
 - [process_mdout](https://biobb-amber.readthedocs.io/en/latest/process.html#module-process.process_mdout) from **biobb_amber.process.process_mdout** 
 - [cpptraj_rms](https://biobb-analysis.readthedocs.io/en/latest/ambertools.html#module-ambertools.cpptraj_rms) from **biobb_analysis.cpptraj.cpptraj_rms**
 - [cpptraj_rgyr](https://biobb-analysis.readthedocs.io/en/latest/ambertools.html#module-ambertools.cpptraj_rgyr) from **biobb_analysis.cpptraj.cpptraj_rgyr**
***

<a id="mdStep1"></a>
### Step 1: Creating portable binary run file to run a free MD simulation

The **free** type of the **simulation_type property** contains the main default parameters to run an **unrestrained MD simulation**:

-  imin = 0;&nbsp;&nbsp;&nbsp;      Run MD (no minimization)
-  ntx = 5;&nbsp;&nbsp;&nbsp;       Read initial coords and vels from restart file
-  cut = 10.0;&nbsp;&nbsp;&nbsp;    Cutoff for non bonded interactions in Angstroms
-  ntr = 0;&nbsp;&nbsp;&nbsp;       No restrained atoms
-  ntc = 2;&nbsp;&nbsp;&nbsp;       SHAKE for constraining length of bonds involving Hydrogen atoms
-  ntf = 2;&nbsp;&nbsp;&nbsp;       Bond interactions involving H omitted
-  ntt = 3;&nbsp;&nbsp;&nbsp;       Constant temperature using Langevin dynamics
-  ig = -1;&nbsp;&nbsp;&nbsp;       Seed for pseudo-random number generator
-  ioutfm = 1;&nbsp;&nbsp;&nbsp;    Write trajectory in netcdf format
-  iwrap = 1;&nbsp;&nbsp;&nbsp;     Wrap coords into primary box
-  nstlim = 5000;&nbsp;&nbsp;&nbsp; Number of MD steps 
-  dt = 0.002;&nbsp;&nbsp;&nbsp;    Time step (in ps)

In this particular example, a short, **5ps-length** simulation (2500 steps) is run, for the sake of time. 

In [35]:
# Import module
from biobb_amber.sander.sander_mdrun import sander_mdrun

# Create prop dict and inputs/outputs
output_free_traj_path = 'sander.free.netcdf'
output_free_rst_path = 'sander.free.rst'
output_free_log_path = 'sander.free.log'

prop = {
    "simulation_type" : 'free',
    "mdin" : { 
        'nstlim' : 2500, # Reducing the number of steps for the sake of time (5ps)
        'ntwx' : 500  # Print coords to trajectory every 500 steps (1 ps)
    }
}

# Create and launch bb
sander_mdrun(input_top_path=output_ions_top_path,
            input_crd_path=output_npt_rst_path,
            output_traj_path=output_free_traj_path,
            output_rst_path=output_free_rst_path,
            output_log_path=output_free_log_path,
            properties=prop)

2021-06-21 17:25:46,865 [MainThread  ] [INFO ]  Creating e6c9f6e3-5dcb-4efc-84b1-98bf3cf137f3 temporary folder
2021-06-21 17:25:46,866 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-06-21 17:41:30,650 [MainThread  ] [INFO ]  sander -O -i e6c9f6e3-5dcb-4efc-84b1-98bf3cf137f3/sander.mdin -p structure.ions.parmtop -c sander.npt.rst -r sander.free.rst -o sander.free.log -x sander.free.netcdf

2021-06-21 17:41:30,651 [MainThread  ] [INFO ]  Exit code 0

2021-06-21 17:41:30,652 [MainThread  ] [INFO ]  Removed: mdinfo, e6c9f6e3-5dcb-4efc-84b1-98bf3cf137f3


0

<a id="mdStep2"></a>
### Step 2: Checking free MD simulation results

Checking results for the final step of the setup process, the **free MD run**. Plotting **Root Mean Square deviation (RMSd)** and **Radius of Gyration (Rgyr)** by time during the **free MD run** step. **RMSd** against the **experimental structure** (input structure of the pipeline) and against the **minimized and equilibrated structure** (output structure of the NPT equilibration step).

In [36]:
# cpptraj_rms: Computing Root Mean Square deviation to analyse structural stability 
#              RMSd against minimized and equilibrated snapshot (backbone atoms)   

# Import module
from biobb_analysis.ambertools.cpptraj_rms import cpptraj_rms

# Create prop dict and inputs/outputs
output_rms_first = pdbCode+'_rms_first.dat'

prop = {
    'mask': 'backbone',
    'reference': 'first'
}

# Create and launch bb
cpptraj_rms(input_top_path=output_ions_top_path,
            input_traj_path=output_free_traj_path,
            output_cpptraj_path=output_rms_first,
            properties=prop)

2021-06-21 17:41:30,685 [MainThread  ] [INFO ]  Not using any container
2021-06-21 17:41:30,858 [MainThread  ] [INFO ]  cpptraj -i 21c3d5d3-24fa-4055-8830-402c8ad417bc/instructions.in

2021-06-21 17:41:30,859 [MainThread  ] [INFO ]  Exit code 0

2021-06-21 17:41:30,860 [MainThread  ] [INFO ]  
CPPTRAJ: Trajectory Analysis. V4.25.6
    ___  ___  ___  ___
     | \/ | \/ | \/ | 
    _|_/\_|_/\_|_/\_|_

| Date/time: 06/21/21 17:41:30
| Available memory: 1.226 GB

INPUT: Reading input from '21c3d5d3-24fa-4055-8830-402c8ad417bc/instructions.in'
  [parm structure.ions.parmtop]
	Reading 'structure.ions.parmtop' as Amber Topology
	Radius Set: modified Bondi radii (mbondi)
  [trajin sander.free.netcdf 1 -1 1]
	Reading 'sander.free.netcdf' as Amber NetCDF
  [center !@H*,1H*,2H*,3H* origin]
    CENTER: Centering coordinates using geometric center of atoms in mask (!@H*,1H*,2H*,3H*) to
	coordinate origin.
  [autoimage]
    AUTOIMAGE: To box center based on center of mass, anchor is first molecule.


0

In [37]:
# cpptraj_rms: Computing Root Mean Square deviation to analyse structural stability 
#              RMSd against experimental structure (backbone atoms)   

# Import module
from biobb_analysis.ambertools.cpptraj_rms import cpptraj_rms

# Create prop dict and inputs/outputs
output_rms_exp = pdbCode+'_rms_exp.dat'

prop = {
    'mask': 'backbone',
    'reference': 'experimental'
}

# Create and launch bb
cpptraj_rms(input_top_path=output_ions_top_path,
            input_traj_path=output_free_traj_path,
            output_cpptraj_path=output_rms_exp,
            input_exp_path=downloaded_pdb, 
            properties=prop)

2021-06-21 17:41:30,880 [MainThread  ] [INFO ]  Not using any container
2021-06-21 17:41:31,092 [MainThread  ] [INFO ]  cpptraj -i cf09d320-e992-4c53-b6f6-b874f023c8c5/instructions.in

2021-06-21 17:41:31,094 [MainThread  ] [INFO ]  Exit code 0

2021-06-21 17:41:31,098 [MainThread  ] [INFO ]  
CPPTRAJ: Trajectory Analysis. V4.25.6
    ___  ___  ___  ___
     | \/ | \/ | \/ | 
    _|_/\_|_/\_|_/\_|_

| Date/time: 06/21/21 17:41:30
| Available memory: 1.211 GB

INPUT: Reading input from 'cf09d320-e992-4c53-b6f6-b874f023c8c5/instructions.in'
  [parm structure.ions.parmtop]
	Reading 'structure.ions.parmtop' as Amber Topology
	Radius Set: modified Bondi radii (mbondi)
  [trajin sander.free.netcdf 1 -1 1]
	Reading 'sander.free.netcdf' as Amber NetCDF
  [center !@H*,1H*,2H*,3H* origin]
    CENTER: Centering coordinates using geometric center of atoms in mask (!@H*,1H*,2H*,3H*) to
	coordinate origin.
  [autoimage]
    AUTOIMAGE: To box center based on center of mass, anchor is first molecule.


0

In [38]:
# Read RMS vs first snapshot data from file 
with open(output_rms_first,'r') as rms_first_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in rms_first_file 
            if not line.startswith(("#","@")) 
        ])
    )

# Read RMS vs experimental structure data from file 
with open(output_rms_exp,'r') as rms_exp_file:
    x2,y2 = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in rms_exp_file
            if not line.startswith(("#","@")) 
        ])
    )
    
trace1 = go.Scatter(
    x = x,
    y = y,
    name = 'RMSd vs first'
)

trace2 = go.Scatter(
    x = x,
    y = y2,
    name = 'RMSd vs exp'
)

data = [trace1, trace2]

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": data,
    "layout": go.Layout(title="RMSd during free MD Simulation",
                        xaxis=dict(title = "Time (ps)"),
                        yaxis=dict(title = "RMSd (Angstrom)")
                       )
}

plotly.offline.iplot(fig)


In [39]:
# cpptraj_rgyr: Computing Radius of Gyration to measure the protein compactness during the free MD simulation 

# Import module
from biobb_analysis.ambertools.cpptraj_rgyr import cpptraj_rgyr

# Create prop dict and inputs/outputs
output_rgyr = pdbCode+'_rgyr.dat'

prop = {
    'mask': 'backbone'
}

# Create and launch bb
cpptraj_rgyr(input_top_path=output_ions_top_path,
            input_traj_path=output_free_traj_path,
            output_cpptraj_path=output_rgyr,
            properties=prop)


2021-06-21 17:41:31,200 [MainThread  ] [INFO ]  Not using any container
2021-06-21 17:41:31,394 [MainThread  ] [INFO ]  cpptraj -i 141d7485-77b6-4b74-b79a-e95469c264b7/instructions.in

2021-06-21 17:41:31,395 [MainThread  ] [INFO ]  Exit code 0

2021-06-21 17:41:31,396 [MainThread  ] [INFO ]  
CPPTRAJ: Trajectory Analysis. V4.25.6
    ___  ___  ___  ___
     | \/ | \/ | \/ | 
    _|_/\_|_/\_|_/\_|_

| Date/time: 06/21/21 17:41:31
| Available memory: 1.207 GB

INPUT: Reading input from '141d7485-77b6-4b74-b79a-e95469c264b7/instructions.in'
  [parm structure.ions.parmtop]
	Reading 'structure.ions.parmtop' as Amber Topology
	Radius Set: modified Bondi radii (mbondi)
  [trajin sander.free.netcdf 1 -1 1]
	Reading 'sander.free.netcdf' as Amber NetCDF
  [center !@H*,1H*,2H*,3H* origin]
    CENTER: Centering coordinates using geometric center of atoms in mask (!@H*,1H*,2H*,3H*) to
	coordinate origin.
  [autoimage]
    AUTOIMAGE: To box center based on center of mass, anchor is first molecule.


0

In [40]:
# Read Rgyr data from file 
with open(output_rgyr,'r') as rgyr_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in rgyr_file 
            if not line.startswith(("#","@")) 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Radius of Gyration",
                        xaxis=dict(title = "Time (ps)"),
                        yaxis=dict(title = "Rgyr (Angstrom)")
                       )
}

plotly.offline.iplot(fig)

<a id="post"></a>
***
## Post-processing and Visualizing resulting 3D trajectory
Post-processing and Visualizing the **protein system** MD setup **resulting trajectory** using **NGL**
- [Step 1](#ppStep1): *Imaging* the resulting trajectory, **stripping out water molecules and ions** and **correcting periodicity issues**.
- [Step 2](#ppStep3): Visualizing the *imaged* trajectory using the *dry* structure as a **topology**. 
***
**Building Blocks** used:
 - [cpptraj_image](https://biobb-analysis.readthedocs.io/en/latest/ambertools.html#module-ambertools.cpptraj_image) from **biobb_analysis.cpptraj.cpptraj_image** 
***

<a id="ppStep1"></a>
### Step 1: *Imaging* the resulting trajectory.
Stripping out **water molecules and ions** and **correcting periodicity issues**  

In [41]:
# cpptraj_image: "Imaging" the resulting trajectory
#                Removing water molecules and ions from the resulting structure

# Import module
from biobb_analysis.ambertools.cpptraj_image import cpptraj_image

# Create prop dict and inputs/outputs
output_imaged_traj = pdbCode+'_imaged_traj.trr'

prop = {
    'mask': 'solute',
    'format': 'trr'
}

# Create and launch bb
cpptraj_image(input_top_path=output_ions_top_path,
            input_traj_path=output_free_traj_path,
            output_cpptraj_path=output_imaged_traj,
            properties=prop)

2021-06-21 17:41:31,452 [MainThread  ] [INFO ]  Not using any container
2021-06-21 17:41:31,635 [MainThread  ] [INFO ]  cpptraj -i 88cc1462-97b8-4a49-84a4-91e7ed796e95/instructions.in

2021-06-21 17:41:31,637 [MainThread  ] [INFO ]  Exit code 0

2021-06-21 17:41:31,638 [MainThread  ] [INFO ]  
CPPTRAJ: Trajectory Analysis. V4.25.6
    ___  ___  ___  ___
     | \/ | \/ | \/ | 
    _|_/\_|_/\_|_/\_|_

| Date/time: 06/21/21 17:41:31
| Available memory: 1.194 GB

INPUT: Reading input from '88cc1462-97b8-4a49-84a4-91e7ed796e95/instructions.in'
  [parm structure.ions.parmtop]
	Reading 'structure.ions.parmtop' as Amber Topology
	Radius Set: modified Bondi radii (mbondi)
  [trajin sander.free.netcdf 1 -1 1]
	Reading 'sander.free.netcdf' as Amber NetCDF
  [center !@H*,1H*,2H*,3H* origin]
    CENTER: Centering coordinates using geometric center of atoms in mask (!@H*,1H*,2H*,3H*) to
	coordinate origin.
  [autoimage]
    AUTOIMAGE: To box center based on center of mass, anchor is first molecule.


0

<a id="ppStep2"></a>
### Step 2: Visualizing the generated dehydrated trajectory.
Using the **imaged trajectory** (output of the [Post-processing step 1](#ppStep1)) with the **dry structure** (output of the [Structure energy minimization - step 5](#minv)) as a topology.

In [48]:
# Show trajectory
view = nglview.show_simpletraj(nglview.SimpletrajTrajectory(output_imaged_traj, output_ambpdb_path), gui=True)
view

NGLWidget(max_frame=4)

In [50]:
from time import sleep
# range number of frames for the animated gif trajectory
for frame in range(0, 5):
    # set frame to update coordinates
    view.frame = frame
    # make sure to let NGL spending enough time to update coordinates
    sleep(0.5)
    view.download_image(filename='trj_image{}.png'.format(frame))
    # make sure to let NGL spending enough time to render before going to next frame
    sleep(2.0)

In [2]:
import moviepy.editor as mpy

# go to folder where the images are stored
template = './trj_image{}.png'
# get all (sorted) image files
imagefiles = [template.format(str(i)) for i in range(0, 4, 1)]

# make a gif file
frame_per_second = 8
im = mpy.ImageSequenceClip(imagefiles, fps=frame_per_second)
im.write_gif('trajectory.gif', fps=frame_per_second)

t:   0%|          | 0/4 [00:00<?, ?it/s, now=None]

MoviePy - Building file trajectory.gif with imageio.


<a id="output"></a>
## Output files

Important **Output files** generated:
 - {{output_ions_pdb_path}}: **System structure** of the MD setup protocol. Structure generated during the MD setup and used in the MD simulation. With hydrogen atoms, solvent box and counterions.  
 - {{output_free_traj_path}}: **Final trajectory** of the MD setup protocol.
 - {{output_free_rst_path}}: **Final checkpoint file**, with information about the state of the simulation. It can be used to **restart** or **continue** a MD simulation.
 - {{output_ions_top_path}}: **Final topology** of the MD system in AMBER Parm7 format.

**Analysis** (MD setup check) output files generated:
 - {{output_rms_first}}: **Root Mean Square deviation (RMSd)** against **minimized and equilibrated structure** of the final **free MD run step**.
 - {{output_rms_exp}}: **Root Mean Square deviation (RMSd)** against **experimental structure** of the final **free MD run step**.
 - {{output_rgyr}}: **Radius of Gyration** of the final **free MD run step** of the **setup pipeline**.
 

***
<a id="questions"></a>

## Questions & Comments

Questions, issues, suggestions and comments are really welcome!

* GitHub issues:
    * [https://github.com/bioexcel/biobb](https://github.com/bioexcel/biobb)

* BioExcel forum:
    * [https://ask.bioexcel.eu/c/BioExcel-Building-Blocks-library](https://ask.bioexcel.eu/c/BioExcel-Building-Blocks-library)
